### Text summarizer with Self trained model / function

<!-- ![hugging face](https://www.thesoftwarereport.com/wp-content/uploads/2023/09/Hugging-Face2.png) -->
Major Steps over covering the model 
- Scrape blog posts (medium, adobe) for net using beautiful soup 
- chunk them into sentences and summarize over these chunks (using the model to output over in text)

##### 1. Importing Dependencies 

In [1]:
import nltk
import os
import re
import math
import operator
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize,word_tokenize
from bs4 import BeautifulSoup # scrap up blog post from the web 
import requests # make over http calls over to web (to scrap up a page)
import json

In [18]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
Stopwords = set(stopwords.words('english'))
wordlemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Harman\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Harman\AppData\Roaming\nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Harman\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


##### 2. making web-scrapper

In [62]:
# URL = "https://hackernoon.com/simplifying-the-crazy-world-of-linux-distros"
URL = "https://hackernoon.com/nvidia-throws-gamers-under-the-bus"

In [63]:
r = requests.get(URL)

In [64]:
soup = BeautifulSoup(r.text, 'html.parser')
results = soup.find_all(['h1', 'p'])
text = [result.text for result in results]
ARTICLE = ' '.join(text)
script_tag = soup.find('script', type='application/ld+json')

if script_tag:
    # Parse JSON data
    json_data = json.loads(script_tag.string)
    
    # Extract articleBody
    article_body = json_data.get('articleBody', '')

    print(article_body)

    ARTICLE = ARTICLE +". " + article_body
else:
    print("JSON-LD script tag not found.")

If you ever needed proof that is no longer the company you think it is, look no further than the company&apos;s most recent GPU Technology Conference (also known as the GTC). The conference was chock-full of announcements, but the big one was of course the unveiling of the Blackwell class of AI chips which are supposedly going to be better than the widely-in-demand, and equally as expensive, Hopper processors that have powered the likes of OpenAI&apos;s ChatGPT. Nvidia It was the demand for the H100 chips that transformed Nvidia into a company worth over $2 trillion, almost overnight in the larger scheme of things, leading some (including HackerNoon) to whether the stock was in a bubble (almost a quarter of HackerNoon readers thought that it was). But, as if Nvidia was out to prove everyone wrong, the company&apos;s last earnings release showed that not only was it still selling the Hopper processors, it was selling of them and would in the near future. question more continue to do so 

In [65]:
print(ARTICLE)

Nvidia Throws Gamers Under the Bus @sheharyarkhan HackerNoon editor. Open to scoops on music, video games, pop culture, and tech.. If you ever needed proof that is no longer the company you think it is, look no further than the company&apos;s most recent GPU Technology Conference (also known as the GTC). The conference was chock-full of announcements, but the big one was of course the unveiling of the Blackwell class of AI chips which are supposedly going to be better than the widely-in-demand, and equally as expensive, Hopper processors that have powered the likes of OpenAI&apos;s ChatGPT. Nvidia It was the demand for the H100 chips that transformed Nvidia into a company worth over $2 trillion, almost overnight in the larger scheme of things, leading some (including HackerNoon) to whether the stock was in a bubble (almost a quarter of HackerNoon readers thought that it was). But, as if Nvidia was out to prove everyone wrong, the company&apos;s last earnings release showed that not onl

#### 3. Text propressing - and implementing TFIDF 

In [14]:
ARTICLE = ARTICLE.replace('.', '.<eos>')
ARTICLE = ARTICLE.replace('?', '?<eos>')
ARTICLE = ARTICLE.replace('!', '!<eos>')

In [21]:
sentences = ARTICLE.split('<eos>')
sentences

['Sign up Sign in Sign up Sign in Automatic Image Captioning With PyTorch Deepesh Garg Follow -- Listen Share “It’s going to be interesting to see how society deals with artificial intelligence, but it will definitely be cool.',
 '” - Colin Angle This is my first open source project .',
 ' I was selected as a Participant for Open Source Contributions at Student Code-in .',
 ' Actually, It was a two months programme where I was selected for contributions to a Computer Vision Project : Image Captioning .',
 ' In this project, I design and train a CNN-RNN (Convolutional Neural Network — Recurrent Neural Network) model for automatically generating image captions.',
 ' In this case, LSTM (Long Short Term Memory), is used which is a special kind of RNN that includes a memory cell, in order to maintain the information for a longer period of time.',
 ' The network is trained on the Microsoft Common Objects in COntext (MS COCO) dataset.',
 ' The image captioning model is displayed below.',
 ' I

In [66]:
def lemmatize_words(words):
    lemmatized_words = []
    for word in words:
       lemmatized_words.append(wordlemmatizer.lemmatize(word))
    return lemmatized_words

In [67]:
def stem_words(words):
    stemmed_words = []
    for word in words:
       stemmed_words.append(stemmer.stem(word))
    return stemmed_words


In [10]:
def remove_special_characters(text):
    regex = r'[^a-zA-Z0-9\s]'
    text = re.sub(regex,'',text)
    return text

In [68]:
tokenized_sentence = sent_tokenize(ARTICLE)
text = remove_special_characters(str(ARTICLE))
text = re.sub(r'\d+', '', text)

In [69]:
text

'Nvidia Throws Gamers Under the Bus sheharyarkhan HackerNoon editor Open to scoops on music video games pop culture and tech If you ever needed proof that is no longer the company you think it is look no further than the companyaposs most recent GPU Technology Conference also known as the GTC The conference was chockfull of announcements but the big one was of course the unveiling of the Blackwell class of AI chips which are supposedly going to be better than the widelyindemand and equally as expensive Hopper processors that have powered the likes of OpenAIaposs ChatGPT Nvidia It was the demand for the H chips that transformed Nvidia into a company worth over  trillion almost overnight in the larger scheme of things leading some including HackerNoon to whether the stock was in a bubble almost a quarter of HackerNoon readers thought that it was But as if Nvidia was out to prove everyone wrong the companyaposs last earnings release showed that not only was it still selling the Hopper pro

In [70]:
tokenized_sentence

['Nvidia Throws Gamers Under the Bus @sheharyarkhan HackerNoon editor.',
 'Open to scoops on music, video games, pop culture, and tech..',
 'If you ever needed proof that is no longer the company you think it is, look no further than the company&apos;s most recent GPU Technology Conference (also known as the GTC).',
 'The conference was chock-full of announcements, but the big one was of course the unveiling of the Blackwell class of AI chips which are supposedly going to be better than the widely-in-demand, and equally as expensive, Hopper processors that have powered the likes of OpenAI&apos;s ChatGPT.',
 'Nvidia It was the demand for the H100 chips that transformed Nvidia into a company worth over $2 trillion, almost overnight in the larger scheme of things, leading some (including HackerNoon) to whether the stock was in a bubble (almost a quarter of HackerNoon readers thought that it was).',
 'But, as if Nvidia was out to prove everyone wrong, the company&apos;s last earnings relea

In [71]:
tokenized_words_with_stopwords = word_tokenize(text)
tokenized_words = [word for word in tokenized_words_with_stopwords if word not in Stopwords]
tokenized_words = [word for word in tokenized_words if len(word) > 1]
tokenized_words = [word.lower() for word in tokenized_words]
tokenized_words = lemmatize_words(tokenized_words)

In [13]:
def freq(words):
    words = [word.lower() for word in words]
    dict_freq = {}
    words_unique = []
    for word in words:
       if word not in words_unique:
           words_unique.append(word)
    for word in words_unique:
       dict_freq[word] = words.count(word)
    return dict_freq


In [72]:
print(tokenized_words)

['nvidia', 'throw', 'gamers', 'under', 'bus', 'sheharyarkhan', 'hackernoon', 'editor', 'open', 'scoop', 'music', 'video', 'game', 'pop', 'culture', 'tech', 'if', 'ever', 'needed', 'proof', 'longer', 'company', 'think', 'look', 'companyaposs', 'recent', 'gpu', 'technology', 'conference', 'also', 'known', 'gtc', 'the', 'conference', 'chockfull', 'announcement', 'big', 'one', 'course', 'unveiling', 'blackwell', 'class', 'ai', 'chip', 'supposedly', 'going', 'better', 'widelyindemand', 'equally', 'expensive', 'hopper', 'processor', 'powered', 'like', 'openaiaposs', 'chatgpt', 'nvidia', 'it', 'demand', 'chip', 'transformed', 'nvidia', 'company', 'worth', 'trillion', 'almost', 'overnight', 'larger', 'scheme', 'thing', 'leading', 'including', 'hackernoon', 'whether', 'stock', 'bubble', 'almost', 'quarter', 'hackernoon', 'reader', 'thought', 'but', 'nvidia', 'prove', 'everyone', 'wrong', 'companyaposs', 'last', 'earnings', 'release', 'showed', 'still', 'selling', 'hopper', 'processor', 'selling

In [73]:
word_freq = freq(tokenized_words)
input_user = int(input('Percentage of information to retain(in percent):'))
no_of_sentences = int((input_user * len(tokenized_sentence))/100)

In [74]:
print(word_freq)

{'nvidia': 15, 'throw': 1, 'gamers': 4, 'under': 1, 'bus': 1, 'sheharyarkhan': 1, 'hackernoon': 4, 'editor': 2, 'open': 1, 'scoop': 1, 'music': 1, 'video': 3, 'game': 3, 'pop': 1, 'culture': 1, 'tech': 8, 'if': 1, 'ever': 1, 'needed': 2, 'proof': 1, 'longer': 2, 'company': 20, 'think': 1, 'look': 1, 'companyaposs': 6, 'recent': 1, 'gpu': 1, 'technology': 5, 'conference': 3, 'also': 2, 'known': 2, 'gtc': 3, 'the': 4, 'chockfull': 1, 'announcement': 2, 'big': 1, 'one': 3, 'course': 2, 'unveiling': 2, 'blackwell': 2, 'class': 1, 'ai': 13, 'chip': 3, 'supposedly': 1, 'going': 1, 'better': 3, 'widelyindemand': 1, 'equally': 1, 'expensive': 1, 'hopper': 2, 'processor': 3, 'powered': 1, 'like': 2, 'openaiaposs': 1, 'chatgpt': 1, 'it': 2, 'demand': 3, 'transformed': 1, 'worth': 1, 'trillion': 1, 'almost': 2, 'overnight': 1, 'larger': 1, 'scheme': 1, 'thing': 1, 'leading': 2, 'including': 1, 'whether': 1, 'stock': 1, 'bubble': 1, 'quarter': 1, 'reader': 1, 'thought': 1, 'but': 3, 'prove': 1, 'e

In [75]:
no_of_sentences

16

In [14]:
def pos_tagging(text):
    pos_tag = nltk.pos_tag(text.split())
    pos_tagged_noun_verb = []
    for word,tag in pos_tag:
        if tag == "NN" or tag == "NNP" or tag == "NNS" or tag == "VB" or tag == "VBD" or tag == "VBG" or tag == "VBN" or tag == "VBP" or tag == "VBZ":
             pos_tagged_noun_verb.append(word)
    return pos_tagged_noun_verb

In [15]:
def tf_score(word,sentence):
    freq_sum = 0
    word_frequency_in_sentence = 0
    len_sentence = len(sentence)
    for word_in_sentence in sentence.split():
        if word == word_in_sentence:
            word_frequency_in_sentence = word_frequency_in_sentence + 1
    tf =  word_frequency_in_sentence/ len_sentence
    return tf

In [16]:
def idf_score(no_of_sentences,word,sentences):
    no_of_sentence_containing_word = 0
    for sentence in sentences:
        sentence = remove_special_characters(str(sentence))
        sentence = re.sub(r'\d+', '', sentence)
        sentence = sentence.split()
        sentence = [word for word in sentence if word.lower() not in Stopwords and len(word)>1]
        sentence = [word.lower() for word in sentence]
        sentence = [wordlemmatizer.lemmatize(word) for word in sentence]
        if word in sentence:
            no_of_sentence_containing_word = no_of_sentence_containing_word + 1
    idf = math.log10(no_of_sentences/no_of_sentence_containing_word)
    return idf

def tf_idf_score(tf,idf):
    return tf*idf

In [32]:
def word_tfidf(dict_freq,word,sentences,sentence):
    word_tfidf = []
    tf = tf_score(word,sentence)
    idf = idf_score(len(sentences),word,sentences)
    tf_idf = tf_idf_score(tf,idf)
    return tf_idf

In [37]:
def sentence_importance(sentence,dict_freq,sentences):
     sentence_score = 0
     sentence = remove_special_characters(str(sentence)) 
     sentence = re.sub(r'\d+', '', sentence)
     pos_tagged_sentence = [] 
     no_of_sentences = len(sentences)
     pos_tagged_sentence = pos_tagging(sentence)
     for word in pos_tagged_sentence:
          if word.lower() not in Stopwords and word not in Stopwords and len(word)>1: 
                word = word.lower()
                word = wordlemmatizer.lemmatize(word)
                sentence_score = sentence_score + word_tfidf(dict_freq,word,sentences,sentence)
     return sentence_score

##### 6. Summarize Text

In [76]:
c = 1
sentence_with_importance = {}
for sent in tokenized_sentence:
    sentenceimp = sentence_importance(sent,word_freq,tokenized_sentence)    
    sentence_with_importance[c] = sentenceimp    
    c = c+1
sentence_with_importance = sorted(sentence_with_importance.items(), key=operator.itemgetter(1),reverse=True)

In [77]:
sentence_with_importance

[(32, 0.1684321913032775),
 (26, 0.1354303984185118),
 (49, 0.11124174943532762),
 (28, 0.10963537564720773),
 (19, 0.10466595934631223),
 (2, 0.09966499136188159),
 (46, 0.09792538955329808),
 (12, 0.09668681608301355),
 (50, 0.09652783713497093),
 (54, 0.09263327861796161),
 (15, 0.09177389492725727),
 (40, 0.0916598974965907),
 (18, 0.08832487175901407),
 (6, 0.08750629725480333),
 (44, 0.08627656553071968),
 (23, 0.08248770930771258),
 (45, 0.08091290472010906),
 (7, 0.08032443182281127),
 (39, 0.08032443182281127),
 (34, 0.0799466893411325),
 (38, 0.07949238458311267),
 (17, 0.07919692744354634),
 (30, 0.0780961598860119),
 (42, 0.07566794302148887),
 (41, 0.07530401386608623),
 (37, 0.07206240963506014),
 (29, 0.06906644678116737),
 (24, 0.06890167706431498),
 (36, 0.06633747831250109),
 (11, 0.06542421335592102),
 (47, 0.0643470030620201),
 (9, 0.06327932387611919),
 (22, 0.062015114578875825),
 (21, 0.05853335375703345),
 (51, 0.057420065217026424),
 (35, 0.057050015962576646),

In [78]:
cnt = 0
summary = []
sentence_no = []
for word_prob in sentence_with_importance:
    if cnt < no_of_sentences:
        sentence_no.append(word_prob[0])
        cnt = cnt+1
    else:
      break
sentence_no.sort()
cnt = 1
for sentence in tokenized_sentence:
    if cnt in sentence_no:
       summary.append(sentence)
    cnt = cnt+1

summary = " ".join(summary)
summary

'Open to scoops on music, video games, pop culture, and tech.. But, as if Nvidia was out to prove everyone wrong, the company&apos;s last earnings release showed that not only was it still selling the Hopper processors, it was selling of them and would in the near future. Worry about supply. , ladies and gentlemen, is the future tech companies had hoped for when they began aggressively hiring during the worldwide pandemic only to find demand normalizing once the world opened up. And therein lies Nvidia&apos;s success. You see, just a day after the GTC began, Jensen Huang held a with investors to tell them the company was poised to win a large chunk of the $250 billion corporations spend on building and maintaining data centers every year given the variety of hardware and software Nvidia has on offer. But as an offshoot of all the research it did in producing better and better graphic cards, it ended up creating tensor cores for real-time deep learning image enhancement and upscaling te

In [79]:
len(ARTICLE), len(summary)

(7685, 2450)

In [80]:
with open('summary.txt', 'w') as f:
    f.write(summary)